In [1]:
from pynq import Overlay
import os
import sys
import numpy as np

HOP_DIR=os.path.abspath("../")
sys.path.insert(0, HOP_DIR)
import pushpush

OVERLAY_DIR=f'{HOP_DIR}/overlays/a_add/'

# Preparation

#### Load the overlay onto the FPGA

In [2]:
ol = Overlay(OVERLAY_DIR + "a_add.bit")

#### Register the overlay with the HoP context

In [3]:
context = pushpush.Context(ol)

# Use

#### Create some python functions

In [4]:
def py_func1() -> int:
    return 58

def py_func2() -> int:
    return 1000

def py_func3() -> int:
    return 100

In [5]:
context.print_all_objects()

hardware:
	a : b32
	add : b32 -> b32 -> b32
python:
cpp:


#### Register functions with HoP context

In [23]:
import importlib as il
if 'context' in locals():
    del context
    il.reload(pushpush)
    pushpush.Context.reloadModules()
    context = pushpush.Context(ol)

In [24]:
a_py = context.register(py_func1, "b32")
b_py = context.register(py_func2, "b32")
c_py = context.register(py_func3, "b32")
d_constant = context.register(1000, 'b32')

#### Get our hardware

In [25]:
a_hw = context.functions['hardware']['a']
add = context.functions['hardware']['add']

#### Use Hardware as a regular function
You can mix hardware, functions, variables

In [27]:
print(add(np.uint32(15), np.uint32(2)))
print(add(np.uint32(15), add(np.uint32(1), np.uint32(1))))

17
17


In [ ]:
x = add(a_hw, a_py)
print(x)
y = add(add(a_py, b_py), add(a_py, d_constant))
print(y)
z = add(a_hw, 6)
print(z)

#### Reduce with hardware

In [ ]:
l = randints(32, size=10)
print(functools.reduce(add, l))
print(functools.reduce(lambda a,b: a + b, l))

#### Imitate Keras Sequential Executor
Tensorflow layers can be hardware, example of defining CNN:

##### Imitate above NN

In [ ]:
import random
class fakeNN:
    def __init__(self, layers):
        self.layers = layers
    
    def singleEpoch(self, inputLayer):
        output = inputLayer
        for l in self.layers:
            leftPercent = random.random()
            rightPercent = 1.0 - leftPercent
            left = int(leftPercent * output)
            right = int(rightPercent * output)
            output = l(left, right)
        return output            

In [ ]:
layers = [add,
          add,
          add]
nn = fakeNN(layers)
print(nn.singleEpoch(32))

# Debug

In [ ]:
import functools 
from numpy.random import default_rng
import time
rng = default_rng()
randints = rng.integers
l = randints(32, size=10)
hw_start = time.perf_counter()
print(functools.reduce(add, l))
hw_end = time.perf_counter()
print(f'hw ttf: {hw_end - hw_start}')
sw_start = time.perf_counter()
print(functools.reduce(lambda a,b: a + b, l))
sw_end = time.perf_counter()
print(f'sw ttf: {sw_end - sw_start}')